Ordered chaining
https://www.philipzucker.com/le_find/

```
tSu          vSw
-----------------
  sig[t] S sig[w]
```

But with the usual restrction that we only need to do inferences that increase something?


ground ordered chaining. maybe there is some path compression that can work here?


https://inria.hal.science/inria-00073205/document Pottier thesis




In [ ]:
# do you best to try and store good chains. https://en.wikipedia.org/wiki/Partially_ordered_set#Derived_notions
# chains are kind of like parent
class LEFind():
  chain0 : list[int]
  chain1 : list[int]

  def union(self, a, b): ...
    # seek lowest common ancestor?

  # .. chain

# biunify kata

Everything I can do with terms
- matching
- unification
- egraphs
- knuth bendix
- subterm


Can i extend to "bi" versions?

- BiProlog . BiKanren

Based on the LEfind, should bi substutuitions be done as
{eq : , upper : [] , lower : []}
Presumably the equality case is more common (?) so should be pulled out. 
`(<= a b)` in minikanren would branch or assert?
bi is the analog of e-unification in many respects.
So look more towards FLP for inspiration?

Would LEFind be useful for a CLP(Set)?




In [ ]:
type bisubst = tuple[dict,dict]
def apply(pat, bisubst): ...
def bimatch(t, pat): ...

# smt egraph

In [1]:
from dataclasses import dataclass
from collections import defaultdict
@dataclass
class LEFind():
    parents : dict
    upper : defaultdict(set)
    lower : defaultdict(set)
    def __init__(self):
        self.parents = {}
        self.upper = defaultdict(set)
        self.lower = defaultdict(set)
    def assert_le(self, x, y): # assert to LEFind that x <= y
        x,y = self.find(x), self.find(y)
        if x == y:
            return
        self.upper[x].add(y)
        self.lower[y].add(x)
        if self.is_le(y, x): # propagate antisymmettry x <= y and y <= x implies x == y
            self.union(x, y)
            for z in self.le_set(x) & self.ge_set(y): # anything between the two is squeezed
                self.union(z, y)
            for z in self.le_set(y) & self.ge_set(x): # anything between the two is squeezed. Is this redundant?
                self.union(z, x)
    def assert_ge(self, x, y): # assert to LEFind that x >= y
        self.assert_le(y, x)
    def union(self, x, y): # assert that x == y
        x, y = self.find(x), self.find(y)
        if x != y:
            self.parents[x] = y # refular union find
            self.upper[y].update(self.upper[x]) # merge upper sets
            self.lower[y].update(self.lower[x]) # merge lower sets
    def find(self, x : int) -> int:
        while x in self.parents:
            x = self.parents[x]
        return x
    # The next 3 functions are very similar. is_le can early stop when it hits y.
    def is_le(self, x, y) -> bool:
        # DFS search for y in upper set of x
        x,y = self.find(x), self.find(y)
        if x == y:
            return True
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop() # invariant is that x is already representative
            for z in self.upper[x]:
                # Is there a way to use lower set for pruning?
                z = self.find(z)   # compression could be updating z in place in upper[x]
                if z == y:
                    return True
                elif z not in seen:
                    seen.add(z)
                    todo.append(z)
        return False
    def le_set(self, x) -> set[int]: # all solutions to x <= ?
        x = self.find(x)
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop()
            for z in self.upper[x]:
                z = self.find(z)
                if z not in seen:
                    seen.add(z)
                    todo.append(z)
        return seen
    def ge_set(self, x) -> set[int]: # all solutions to x >= ?
        x = self.find(x)
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop()
            for z in self.lower[x]:
                z = self.find(z)
                if z not in seen:
                    seen.add(z)
                    todo.append(z)
        return seen

Hmm. You know, if there is no further interpretation of le, Having the solver backed version does not help anything.

There is also an issue that I don't really know how to encode a model of undefinedness into SMT.
The relational model? That's more like the contextual egraph right?
The option model?



In [ ]:
from kdrag.solvers.egraph import EGraph

class LEGraph(EGraph):
    def __init__(self):
        super().__init__()
        self.lefind = LEFind()
        self.le_prim = smt.Function("le", smt.IntSort(), smt.IntSort(), smt.BoolSort())
        self.le = smt.TransitiveClosure(self.le_prim)
    
    def assert_le(self, x, y):
        self.lefind.assert_le(x, y)
        self.solver.add(self.le_prim(x, y))

    def is_le(self, x, y):
        return self.lefind.is_le(x, y)
        with self.solver:
            self.solver.add(smt.Not(self.le(x, y)))
            return self.solver.check() == smt.unsat
    def le_set(self, x):
        return self.lefind.le_set(x)





In [ ]:
@dataclass
class EGraph():
    lefind : LEFind
    enodes : dict

